## Importing dependencies and environmental variables

In [1]:
# Dependencies
import csv as csv
import http.client 
import json
import numpy as np
import os
import pandas as pd
import pprint
import psycopg2
import requests
import sqlalchemy
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
from selenium import webdriver
from sodapy import Socrata
from splinter import Browser
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, Float, Date

In [2]:
# Loading homeowrk7.env so that the environment variables can be used in the engine object url
load_dotenv("project3.env")
# Initializing variables to hold each environmet varaible
username=os.environ.get("USERNAME")
password=os.environ.get("PASSWORD")
host=os.environ.get("HOST")
port=os.environ.get("PORT")
database=os.environ.get("DATABASE")
google_key = os.environ.get('GOOGLE_API_KEY')
yelp_client_id = os.environ.get('YELP_CLIENT_ID')
yelp_key = os.environ.get('YELP_API_KEY')
sf_data_key = os.environ.get('SFDATAAPPTOKEN')
acs_5yr_key = os.environ.get('ACS_5YR_KEY')

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Reading in .csv

In [4]:
housing_data = pd.read_csv("data/Sale_Prices_Zip.csv")
zip_codes = pd.read_csv("data/zipcodes.csv")

## Cleaning housing_data to only include data for the zip codes we will be using to train our model

In [5]:
zip_code_list = []
for row in zip_codes.iterrows():
    zip_code_list.append(str(row[1][0]))

In [6]:
housing_data = housing_data[housing_data["RegionName"].isin(zip_code_list)].fillna(0)

In [7]:
housing_data

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09
9,97564,94109,California,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1232600.0,1140400.0,1040900.0,998700.0,1052500.0,1044300.0,1083600,1159300,1181200,1056500,1197300,1174000,1282500,1098100,1099400,1026300,1052100,1063500,1101200,1042000,1000500.0,961700.0,896600.0,980400.0,962200.0,1033900.0,1013300.0,1045700.0,1076400.0,1127300.0,0.0
48,97565,94110,California,49,707700.0,743600.0,730100.0,744500.0,736000.0,800900.0,802800.0,767300.0,692300.0,710200.0,688700.0,672000.0,620100.0,644700.0,646000.0,657800.0,687700.0,728600.0,760500.0,744500.0,771300.0,771100.0,677400.0,622800.0,599400.0,689400.0,709800.0,678000.0,701200.0,712100.0,752100.0,716400.0,746800.0,746500.0,713500.0,669700.0,683200.0,747200.0,761400.0,735000.0,726500.0,704100.0,709900.0,701100.0,701500.0,715100.0,664900.0,670800.0,660300.0,726300.0,716000.0,726500.0,753900.0,814100.0,872000.0,857600.0,860700.0,823400.0,823800.0,777800.0,781500.0,800600.0,878600.0,917400.0,948500.0,940300.0,964900.0,961000.0,1014700.0,1013000.0,1035900.0,1014600.0,1004700.0,987200.0,1004900.0,1062100.0,1081300.0,1088600.0,1044700.0,1063700.0,1093600.0,1156200.0,1186500.0,1212900.0,1254400.0,1288900.0,1225400.0,1169000.0,1175900.0,1248500.0,1264000.0,1246600.0,1239600.0,1273600.0,1274200.0,1272900.0,1269200.0,1296200.0,1294200.0,1287700.0,1273100.0,1331200.0,1370300.0,1400700.0,1338800.0,1303000.0,1272400.0,1205100.0,1237200.0,1215900.0,1335200.0,1352900.0,1419200.0,1378500.0,1396200,1406400,1447400,1454600,1526800,1540900,1560500,1516300,1517200,1501000,1455100,1469100,1489900,1479500,1461100.0,1433800.0,1402900.0,1402900.0,1400400.0,1499000.0,1467000.0,1469700.0,1512000.0,1602500.0,0.0
50,96015,90034,California,51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,892200.0,846500.0,838100.0,865900.0,891500.0,864600.0,833200.0,821700.0,826800.0,811000.0,805000.0,943000.0,972700.0,1059600.0,952300.0,882000.0,817400,884800,1063400,1090800,978600,982500,984200,1067500,1010000,952000,949200,975800,1049600,1049500,1029100.0,1070600.0,1180400.0,1270200.0,1319600.0,1230600.0,1247600.0,1213700.0,1228800.0,1148800.0,1141700.0
100,96047,90066,Calif

In [ ]:
annual_housing_data = 